# IP Project

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"]=15,10 #Set plot size
plt.rcParams.update({'font.size': 16}) #set plot size font
plt.rcParams.update({"axes.grid" : True, "grid.color": "lightgrey"}) #Add grid lines to plot

### Black Box

In [2]:
cols = ['t','V_ms-1','V_knots','Dist']
BB = pd.DataFrame(columns = cols)
BB

,t,V_ms-1,V_knots,Dist


### Other Functions

In [3]:
def Knots(MS):
    return MS / 0.5144

def MS(Knots):
    return Knots * 0.5144

#### Aircraft Parameters

In [4]:
mass = 79000
AbortSpeed = 120 #In Knots

T1 = 106.8 * 1000 #Engine Thrust A320-200 with 2* CFM International LEAP 1A at T/O power (106.80 kN)    ~~TBC~~
T2 = 106.8 * 1000

Decel = 6 #Deceleration of Max Brakes 6ms-2  ~~TBC~~

In [5]:
V = 0
t = 0
t_step = 1e-4

## Ground Roll

Accelerating along Runway with CONSTANT Thrust


In [6]:
L = 0  # Ground Roll (Zero Lift) {for now}
D_i = 0  # Zero Lift -> Drag = 0   {for now}
mu = 0  # Zero Friction {for now}
W = 9.81 * mass
D_f = mu * (L - W)    #Friction Drag

# N2 : F = m a
a = (T1 + T2 - D_i - D_f)/mass

In [ ]:
%%time

L = 0  # Ground Roll (Zero Lift) {for now}
D_i = 0  # Zero Lift -> Drag = 0   {for now}
mu = 0  # Zero Friction {for now}
W = 9.81 * mass
D_f = mu * (L - W)    #Friction Drag

# N2 : F = m a
a = (T1 + T2 - D_i - D_f)/mass

#Set initial condition
V = 0
t = 0
s = 0

#Calculate the acceleration stage
while Knots(V) < AbortSpeed:
    V += a * t_step
    s += V*t_step + 0.5*a*t_step**2
    
    DataRow = pd.DataFrame({'t': [t], 'V_ms-1': [V], 'V_knots': [Knots(V)],'Dist':[s]})
    BB = BB.append(DataRow)

    t += t_step
        
print('Run aborted at {:.6f}'.format(t))

In [ ]:
BB

In [ ]:
%%time

#Calculate the braking stage
while V > 0:
    V -= Decel * t_step
    if V < 0:
        V = 0
    s += V*t_step + 0.5*a*t_step**2
    
    DataRow = pd.DataFrame({'t': [t], 'V_ms-1': [V], 'V_knots': [Knots(V)],'Dist':[s]})
    BB = BB.append(DataRow)

    t += t_step

#Output time that aircraft stops and distance travelled
print('Run stopped at {:.6f}'.format(t))
print('Run length     {:.6f}'.format(BB.tail(1)['Dist'][0]))

In [ ]:
BB

In [ ]:
print('Run stopped at {:.6f}'.format(t))
print('Run length     {:.6f}'.format(BB.tail(1)['Dist'][0]))

In [ ]:
plt.plot(BB['t'],BB['V_ms-1'])
plt.grid()

In [ ]:
plt.plot(BB['t'],BB['Dist'])
plt.grid()

### Notes to self
 - Speed of code execution slow and increasing with smaller t_step
    - Swap to use of standard numpy array or look at Vaex (https://vaex.io/docs/index.html)
 - Use of ODEs rather than euler stepping
 